# RNN(Recurrent Neural Network) implementation

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
docs = ['recurrent neural network',
		'neural network',
		'artificial neural',
		'connections between nodes',
		'can create a cycle',
		'allowing output',
		'some nodes to affect subsequent',
		'exhibit temporal',
		'dynamic behavior',
		'type of Neural Network',
    'affect subsequent']

In [4]:
tokenizer = Tokenizer(oov_token = '<nothing>')
tokenizer.fit_on_texts(docs)

In [5]:
tokenizer.word_index

{'<nothing>': 1,
 'neural': 2,
 'network': 3,
 'nodes': 4,
 'affect': 5,
 'subsequent': 6,
 'recurrent': 7,
 'artificial': 8,
 'connections': 9,
 'between': 10,
 'can': 11,
 'create': 12,
 'a': 13,
 'cycle': 14,
 'allowing': 15,
 'output': 16,
 'some': 17,
 'to': 18,
 'exhibit': 19,
 'temporal': 20,
 'dynamic': 21,
 'behavior': 22,
 'type': 23,
 'of': 24}

In [6]:
tokenizer.word_counts

OrderedDict([('recurrent', 1),
             ('neural', 4),
             ('network', 3),
             ('artificial', 1),
             ('connections', 1),
             ('between', 1),
             ('nodes', 2),
             ('can', 1),
             ('create', 1),
             ('a', 1),
             ('cycle', 1),
             ('allowing', 1),
             ('output', 1),
             ('some', 1),
             ('to', 1),
             ('affect', 2),
             ('subsequent', 2),
             ('exhibit', 1),
             ('temporal', 1),
             ('dynamic', 1),
             ('behavior', 1),
             ('type', 1),
             ('of', 1)])

# TEXT -> VECTOR
- Assigning a number to each unique word appears in the text.

In [7]:
sequences = tokenizer.texts_to_sequences(docs)
sequences

[[7, 2, 3],
 [2, 3],
 [8, 2],
 [9, 10, 4],
 [11, 12, 13, 14],
 [15, 16],
 [17, 4, 18, 5, 6],
 [19, 20],
 [21, 22],
 [23, 24, 2, 3],
 [5, 6]]

In [8]:
len(sequences[0]) == len(sequences[2])

False

## Zero Padding

- It convert all the `input sequences` to same length to maintain uniformity in vectors length.

In [9]:
from keras.utils import pad_sequences

In [10]:
sequences = pad_sequences(sequences, padding = 'post')
sequences

array([[ 7,  2,  3,  0,  0],
       [ 2,  3,  0,  0,  0],
       [ 8,  2,  0,  0,  0],
       [ 9, 10,  4,  0,  0],
       [11, 12, 13, 14,  0],
       [15, 16,  0,  0,  0],
       [17,  4, 18,  5,  6],
       [19, 20,  0,  0,  0],
       [21, 22,  0,  0,  0],
       [23, 24,  2,  3,  0],
       [ 5,  6,  0,  0,  0]], dtype=int32)

In [11]:
len(sequences[0]) == len(sequences[2])

True

# Sentiment Analysis

In [12]:
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense, Flatten, SimpleRNN, Embedding

In [13]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
len(X_train)

25000

In [15]:
max_len = 0

for i in range(len(X_train)):
    if len(X_train[i]) > max_len:
        max_len = len(X_train[i])

max_len

2494

In [16]:
# Let's pad the data to make equal sizes vectors in data

X_train = pad_sequences(X_train, padding = 'post', maxlen = 50)
X_test = pad_sequences(X_test, padding = 'post', maxlen = 50)

In [17]:
print(len(X_train[0]))
print(len(X_test[0]))

50
50


## RNN implementation

Without Embedding layer

In [18]:
model = Sequential()
model.add(SimpleRNN(32, input_shape = (50,1), return_sequences = False))      # it means it won't return the RNN's intermediate O/P if 'False'
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,121 (4.38 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.5072 - loss: 0.7073 - val_accuracy: 0.4973 - val_loss: 0.6957
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.4997 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss: 0.6946
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5130 - loss: 0.6924 - val_accuracy: 0.5024 - val_loss: 0.6945
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5035 - loss: 0.6929 - val_accuracy: 0.5036 - val_loss: 0.6946
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5109 - loss: 0.6927 - val_accuracy: 0.5019 - val_loss: 0.6949


# RNN implementation

With Embedding layer

In [21]:
em_model = Sequential()
em_model.add(Embedding(10000, output_dim = 2, input_length = 50))     # vocab size = 10k, embedding dim = 2
em_model.add(SimpleRNN(32, return_sequences = False))                 # it means it won't return the RNN's intermediate O/P if 'False'
em_model.add(Dense(1, activation = 'sigmoid'))
em_model.build(input_shape=(None, 50))                                # explicitly build with sequence length 50    # added extra line to run the code

em_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 2)          │        20,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 32)             │         1,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,153 (82.63 KB)

 Trainable params: 21,153 (82.63 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs =1, validation_data = (X_test, y_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.5076 - loss: 0.6931 - val_accuracy: 0.5020 - val_loss: 0.6944


In [23]:
X_train.shape

(25000, 50)

In [28]:
y_pred = model.predict(X_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [29]:
y_pred

array([[0.497114],
       [0.497114],
       [0.497114],
       ...,
       [0.497114],
       [0.497114],
       [0.497114]], dtype=float32)

## NOTE:

- Transforming `y_pred` into `0` and `1` represents for negative and positive sentiments for reviews.

In [31]:
import numpy as np
y_pred_tr = np.argmax(y_pred, axis = 1)
y_pred_tr

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
em_y_pred = em_model.predict(X_test)
em_y_pred

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


array([[0.48252586],
       [0.4732547 ],
       [0.55296093],
       ...,
       [0.52063006],
       [0.5207716 ],
       [0.5169469 ]], dtype=float32)

In [32]:
em_y_pred_tr = np.argmax(em_y_pred, axis = 1)
em_y_pred_tr

array([0, 0, 0, ..., 0, 0, 0])

# Performance metrics:

## For model without embedding:

In [35]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [37]:
cm = confusion_matrix(y_test, y_pred_tr)
ac_sc = accuracy_score(y_test, y_pred_tr)
cl_report = classification_report(y_test, y_pred_tr)

print("Confusion matrix: \n", cm)
print("\nAccuracy score: ", ac_sc)
print("\nClassification report: \n", cl_report)

Confusion matrix: 
 [[12500     0]
 [12500     0]]

Accuracy score:  0.5

Classification report: 
               precision    recall  f1-score   support

           0       0.50      1.00      0.67     12500
           1       0.00      0.00      0.00     12500

    accuracy                           0.50     25000
   macro avg       0.25      0.50      0.33     25000
weighted avg       0.25      0.50      0.33     25000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## For model with embedding layer

In [38]:
cm = confusion_matrix(y_test, em_y_pred_tr)
ac_sc = accuracy_score(y_test, em_y_pred_tr)
cl_report = classification_report(y_test, em_y_pred_tr)

print("Confusion matrix: \n", cm)
print("\nAccuracy score: ", ac_sc)
print("\nClassification report: \n", cl_report)

Confusion matrix: 
 [[12500     0]
 [12500     0]]

Accuracy score:  0.5

Classification report: 
               precision    recall  f1-score   support

           0       0.50      1.00      0.67     12500
           1       0.00      0.00      0.00     12500

    accuracy                           0.50     25000
   macro avg       0.25      0.50      0.33     25000
weighted avg       0.25      0.50      0.33     25000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Analysis

- We got same result with and without Embedding layer in an RNN model.